In [12]:
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from scantools import run_manual_anonymization
from scantools.utils.io import read_image
from scantools.viz.image import plot_images

from redact.v3.data_models import JobLabels
from scantools.proc.anonymization import BrighterAIAnonymizer
anon = BrighterAIAnonymizer('dummy')
image_shape = (1920, 1440, 3)

def detection_is_big(face, image_shape):
    mx, my, Mx, My = face.bounding_box
    area_ratio = (Mx - mx + 1) * (My - my + 1) / (image_shape[0] * image_shape[1])
    return face.score >= 0.40 and area_ratio > 0.04

def plot_anon(path, faces, show_all=False):
    image = read_image(path)
    plot_images([image])
    for face in faces:
        is_valid = anon.face_is_valid(face, image.shape)
        if not show_all and not is_valid:
            continue
        mx, my, Mx, My = face.bounding_box
        w = Mx - mx
        h = My - my
        if face.score is None:
            color = 'blue'
        elif is_valid:
            color = 'lime'
        else:
            color = 'red'
        plt.gca().add_patch(Rectangle((mx, my), w, h, fc='none', ec=color))
        if face.score is not None:
            plt.text(mx, my-5, f'{face.score*100:.0f}%', c=color, fontsize=9)
    
import collections    
def get_counts(root, session_id=None, prefix=None):
    if session_id is None:
        seqs = [p.name for p in (root / 'sessions').iterdir()]
    else:
        seqs = (root/'sessions'/session_id/'proc/subsessions.txt').read_text().split('\n')
    counter = collections.Counter()
    for seq in seqs:
        if prefix is not None and not seq.startswith(prefix):
            continue
        d = root / 'anonymization'
        if session_id is not None:
            d = d / session_id
        for p in (d / seq).glob('**/labels.json'):
            labels = JobLabels.parse_file(p)
            counter[seq] += sum(anon.face_is_valid(ff, image_shape) for f in labels.frames for ff in f.faces)
    return sorted(counter.items(), key=lambda x: -x[1])

In [2]:
# Find sequences with many detected faces
scene = 'LIN'
root = Path('/media/SSD2/lamar/captures/release/', scene)
get_counts(root)

[('ios_2022-06-20_15.30.34', 5382)]

In [ ]:
# CAB
session_id = 'ios_2022-01-12_16.27.04'
session_id = 'ios_2022-01-20_14.30.40'

# LIN
session_id = 'ios_2022-06-20_15.30.34'

image_paths = []
face_labels = []
anon_dir = 'anonymization'  # or 'anonymization_sequential'
for p in (root / anon_dir / session_id).glob('**/labels.json'):
    ipaths = (p.parent/'list.txt').read_text().split('\n')
    image_paths.extend(ipaths)
    labels = JobLabels.parse_file(p)
    face_labels.extend([frame.faces for frame in labels.frames])
    assert len(ipaths) == len(labels.frames)
# indices = [i for i, faces in enumerate(face_labels) if faces]
# indices = [i for i, faces in enumerate(face_labels) if any(anon.face_is_valid(f, image_shape) for f in faces)]
indices = [i for i, faces in enumerate(face_labels) if any(detection_is_big(f, image_shape) for f in faces)]
image_paths = [image_paths[i] for i in indices]
face_labels = [face_labels[i] for i in indices]
print(len(indices))

start = 0  # update
skip = 1  # update
num = 0
for subpath, faces in zip(image_paths[start::skip], face_labels[start::skip]):
    if not faces:
        continue
    impath = root / 'sessions' / session_id / 'raw_data' / subpath
    plot_anon(impath, faces, show_all=True)
    plt.gca().set_title(session_id+'/'+Path(subpath).stem, fontsize=9)
    plt.show()
    num += 1
    if num >= 70:  # update
        break

# Find sequences with many faces in previous annotations

In [37]:
root2 = Path('/media/SSD2/lamar/captures/release_eval/', scene)
session_id = 'map_min'
get_counts(root2, session_id, 'ios_')

[('ios_2022-07-03_14.07.18_001', 371),
 ('ios_2022-01-20_14.30.40_000', 125),
 ('ios_2022-01-20_14.51.09_000', 80),
 ('ios_2022-06-22_13.33.34_002', 64),
 ('ios_2022-01-20_15.01.57_000', 56),
 ('ios_2022-06-20_16.02.21_001', 47),
 ('ios_2021-06-02_14.21.49_000', 46),
 ('ios_2021-06-02_15.00.23_005', 34),
 ('ios_2021-06-02_15.00.23_000', 26),
 ('ios_2022-01-12_15.50.47_001', 24),
 ('ios_2022-02-27_17.56.28_000', 21),
 ('ios_2021-06-02_15.00.23_001', 16),
 ('ios_2022-07-03_13.26.22_001', 13),
 ('ios_2022-01-20_16.13.52_000', 9),
 ('ios_2022-06-20_18.27.53_000', 8),
 ('ios_2021-06-03_12.16.58_000', 5),
 ('ios_2021-06-02_14.48.13_000', 2),
 ('ios_2022-06-20_16.07.07_002', 2),
 ('ios_2022-02-27_17.44.49_000', 1),
 ('ios_2021-06-02_15.00.23_004', 0),
 ('ios_2022-06-22_13.33.34_000', 0)]

# Check previous annotations

In [ ]:
root2 = Path('/media/SSD2/lamar/captures/release_eval/', scene)
session_id = 'map_min'
seq_id = 'ios_2022-01-20_14.30.40_000'

# LIN
seq_id = 'ios_2022-06-20_15.30.34_000'
seq_id = 'ios_2022-06-25_20.28.02_000'

image_paths = []
face_labels = []
for p in (root2 / 'anonymization' / session_id).glob('**/labels.json'):
    ipaths = (p.parent/'list.txt').read_text().split('\n')
    ipaths = [i[6:-1] for i in ipaths if i]
    image_paths.extend(ipaths)
    labels = JobLabels.parse_file(p)
    face_labels.extend([frame.faces for frame in labels.frames])
    assert len(ipaths) == len(labels.frames)
    assert len(image_paths) == len(face_labels)
indices = [i for i, faces in enumerate(face_labels) if any(anon.face_is_valid(f, image_shape) for f in faces)]
indices = [i for i in indices if seq_id in image_paths[i]]
image_paths = [image_paths[i] for i in indices]
face_labels = [face_labels[i] for i in indices]
print(len(image_paths))

num = 0
start = 200
skip = 2
for impath, faces in zip(image_paths[start::skip], face_labels[start::skip]):
    if not faces:
        continue
    plot_anon(impath.replace('ms_eth_dataset/datasets', 'lamar'), faces)
    plt.show()
    num += 1
    if num >= 50:
        break